# Automatic Stimuli Creation for Degrading Visual Information about Articulation or Mouthing
<br>
<div align="center">Wim Pouw (wim.pouw@donders.ru.nl)</div>

<img src="Images/envision_banner.png" alt="isolated" width="300"/>

## Info documents

This python notebook runs you through the procedure of taking videos as inputs with a single person in the video, and outputting the 1 outputs of the kinematic timeseries, and optionally masking video with facial, hand, and arm kinematics ovelayen.

The masked-piper tool is a simple but effective modification of the the Holistic Tracking by Google's Mediapipe so that we can use it as a CPU-based light weigth tool to mask your video data while maintaining background information, and also preserving information about body kinematics. 

* location Repository:  https://github.com/WimPouw/envisionBOX_modulesWP/tree/main/Mediapipe_Optional_Masking

* location Jupyter notebook: https://github.com/WimPouw/envisionBOX_modulesWP/blob/main/MultimodalMerging/Masking_Mediapiping.ipynb

Current Github: https://github.com/WimPouw/TowardsMultimodalOpenScience

## Additional information backbone of the tool (Mediapipe Holistic Tracking)
https://google.github.io/mediapipe/solutions/holistic.html

## Citation of mediapipe
citation: Lugaresi, C., Tang, J., Nash, H., McClanahan, C., Uboweja, E., Hays, M., ... & Grundmann, M. (2019). Mediapipe: A framework for building perception pipelines. arXiv preprint arXiv:1906.08172.

## Citation of masked piper
* citation: Owoyele, B., Trujillo, J., De Melo, G., & Pouw, W. (2022). Masked-Piper: Masking personal identities in visual recordings while preserving multimodal information. SoftwareX, 20, 101236. 
* Original Repo: https://github.com/WimPouw/TowardsMultimodalOpenScience

## Modification that is the basis of this tool
Our modification of the Mediapipe tool is using the body sillhoette to distinguish the background from the body contained in the video, then track the body, and create new video that only keeps the background, masks the body, and overlays the kinematics back onto the mask. We further modify the original code so that timeseries are produced that provide all the kinematic information per frame over time.

## Use
Make sure to install all the packages in requirements.txt. Then move your videos that you want to mask into the input folder. Then run this code, which will loop through all the videos contained in the input folder; and saves all the results in the output folders.

Please use, improve and adapt as you see fit.

Team: Babajide Owoyele, James Trujillo, Gerard de Melo, Wim Pouw (wim.pouw@donders.ru.nl)


In [1]:
#load in required packages
import mediapipe as mp #mediapipe
import cv2 #opencv
import math #basic operations
import numpy as np #basic operations
import pandas as pd #data wrangling
import csv #csv saving
import os #some basic functions for inspecting folder structure etc.

#list all videos in input_videofolder
from os import listdir
from os.path import isfile, join
mypath = "./Input_Videos/" #this is your folder with (all) your video(s)
vfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))] #loop through the filenames and collect them in a list
#time series output folder
inputfol = "./Input_Videos/"
outputf_mask = "./Output_Videos/"
outtputf_ts = "./Output_TimeSeries/"

#check videos to be processed
print("The following folder is set as the output folder where all the pose time series are stored")
print(os.path.abspath(outtputf_ts))
print("\n The following folder is set as the output folder for saving the masked videos ")
print(os.path.abspath(outputf_mask))
print("\n The following video(s) will be processed for masking: ")
print(vfiles)

The following folder is set as the output folder where all the pose time series are stored
d:\Research_projects\envisionBOX_modulesWP\MouthRegionMasking\Output_TimeSeries

 The following folder is set as the output folder for saving the masked videos 
d:\Research_projects\envisionBOX_modulesWP\MouthRegionMasking\Output_Videos

 The following video(s) will be processed for masking: 
['DOLFIJN.mp4', 'ETEN.mp4', 'NULL.mp4', 'OCHTEND.mp4', 'OLIFANT.mp4', 'RIETJE.mp4']


In [2]:
#initialize modules and functions

#load in mediapipe modules
mp_holistic = mp.solutions.holistic
# Import drawing_utils and drawing_styles.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

##################FUNCTIONS AND OTHER VARIABLES
#landmarks 33x that are used by Mediapipe (Blazepose)
markersbody = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_OUTER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
          'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 
          'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
          'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
          'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']

markershands = ['LEFT_WRIST', 'LEFT_THUMB_CMC', 'LEFT_THUMB_MCP', 'LEFT_THUMB_IP', 'LEFT_THUMB_TIP', 'LEFT_INDEX_FINGER_MCP',
              'LEFT_INDEX_FINGER_PIP', 'LEFT_INDEX_FINGER_DIP', 'LEFT_INDEX_FINGER_TIP', 'LEFT_MIDDLE_FINGER_MCP', 
               'LEFT_MIDDLE_FINGER_PIP', 'LEFT_MIDDLE_FINGER_DIP', 'LEFT_MIDDLE_FINGER_TIP', 'LEFT_RING_FINGER_MCP', 
               'LEFT_RING_FINGER_PIP', 'LEFT_RING_FINGER_DIP', 'LEFT_RING_FINGER_TIP', 'LEFT_PINKY_FINGER_MCP', 
               'LEFT_PINKY_FINGER_PIP', 'LEFT_PINKY_FINGER_DIP', 'LEFT_PINKY_FINGER_TIP',
              'RIGHT_WRIST', 'RIGHT_THUMB_CMC', 'RIGHT_THUMB_MCP', 'RIGHT_THUMB_IP', 'RIGHT_THUMB_TIP', 'RIGHT_INDEX_FINGER_MCP',
              'RIGHT_INDEX_FINGER_PIP', 'RIGHT_INDEX_FINGER_DIP', 'RIGHT_INDEX_FINGER_TIP', 'RIGHT_MIDDLE_FINGER_MCP', 
               'RIGHT_MIDDLE_FINGER_PIP', 'RIGHT_MIDDLE_FINGER_DIP', 'RIGHT_MIDDLE_FINGER_TIP', 'RIGHT_RING_FINGER_MCP', 
               'RIGHT_RING_FINGER_PIP', 'RIGHT_RING_FINGER_DIP', 'RIGHT_RING_FINGER_TIP', 'RIGHT_PINKY_FINGER_MCP', 
               'RIGHT_PINKY_FINGER_PIP', 'RIGHT_PINKY_FINGER_DIP', 'RIGHT_PINKY_FINGER_TIP']
facemarks = [str(x) for x in range(478)] #there are 478 points for the face mesh (see google holistic face mesh info for landmarks)

print("Note that we have the following number of pose keypoints for markers body")
print(len(markersbody))

print("\n Note that we have the following number of pose keypoints for markers hands")
print(len(markershands))

print("\n Note that we have the following number of pose keypoints for markers face")
print(len(facemarks ))

#set up the column names and objects for the time series data (add time as the first variable)
markerxyzbody = ['time']
markerxyzhands = ['time']
markerxyzface = ['time']

for mark in markersbody:
    for pos in ['X', 'Y', 'Z', 'visibility']: #for markers of the body you also have a visibility reliability score
        nm = pos + "_" + mark
        markerxyzbody.append(nm)
for mark in markershands:
    for pos in ['X', 'Y', 'Z']:
        nm = pos + "_" + mark
        markerxyzhands.append(nm)
for mark in facemarks:
    for pos in ['X', 'Y', 'Z']:
        nm = pos + "_" + mark
        markerxyzface.append(nm)

#check if there are numbers in a string
def num_there(s):
    return any(i.isdigit() for i in s)

#take some google classification object and convert it into a string
def makegoginto_str(gogobj):
    gogobj = str(gogobj).strip("[]")
    gogobj = gogobj.split("\n")
    return(gogobj[:-1]) #ignore last element as this has nothing

#make the stringifyd position traces into clean numerical values
def listpostions(newsamplemarks):
    newsamplemarks = makegoginto_str(newsamplemarks)
    tracking_p = []
    for value in newsamplemarks:
        if num_there(value):
            stripped = value.split(':', 1)[1]
            stripped = stripped.strip() #remove spaces in the string if present
            tracking_p.append(stripped) #add to this list  
    return(tracking_p)

Note that we have the following number of pose keypoints for markers body
33

 Note that we have the following number of pose keypoints for markers hands
42

 Note that we have the following number of pose keypoints for markers face
478


## Main procedure Masked-Piper
The following chunk of code loops through all the videos you have loaded into the input folder, then assess each frame for body poses, extract kinematic info, masks the body in a new frame that keeps the background, projects the kinematic info on the mask, and stores the kinematic info for that frame into the time series .csv for the hand + body + face.

In [13]:
# do you want to apply masking?
masking = True
blur_kernel_size = 111  # Adjust this value to change blur intensity
opacity = 1  # 0.0 is fully transparent, 1.0 is fully opaque

# Mouth landmarks for masking
MOUTH_LANDMARKS = [192, 206, 2, 426, 436, 434, 431, 211]

#We will now loop over all the videos that are present in the video file
for vidf in vfiles:
    print("We will now process video:")
    print(vidf)
    print("This is video number " + str(vfiles.index(vidf))+ " of " + str(len(vfiles)) + " videos in total")
    
    videoname = vidf
    videoloc = inputfol + videoname
    capture = cv2.VideoCapture(videoloc)
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    samplerate = capture.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(outputf_mask+videoname, fourcc, 
                         fps = samplerate, frameSize = (int(frameWidth), int(frameHeight)))

    time = 0
    tsbody = [markerxyzbody]
    tshands = [markerxyzhands]
    tsface = [markerxyzface]
    
    with mp_holistic.Holistic(
            static_image_mode=False, enable_segmentation=True, refine_face_landmarks=True) as holistic:
        while (True):
            ret, image = capture.read()
            if ret == True:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)
                
                h, w, c = image.shape
                if np.all(results.face_landmarks) != None:
                    original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    
                    if masking:
                        # Create mask for mouth area
                        mask = np.zeros((h, w), dtype=np.uint8)
                        landmarks = results.face_landmarks.landmark
                        
                        # Get mouth area points
                        mouth_points = np.array([(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) 
                                               for idx in MOUTH_LANDMARKS], dtype=np.int32)
                        
                        # Fill mouth polygon
                        cv2.fillPoly(mask, [mouth_points], 255)
                        
                        # Apply Gaussian blur to the mouth area
                        blur_region = cv2.GaussianBlur(original_image, (blur_kernel_size, blur_kernel_size), 0)
                        
                        # Blend original and blurred image based on mask and opacity
                        mask_3channel = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) / 255.0
                        original_image = (original_image * (1 - mask_3channel * opacity) + 
                                        blur_region * (mask_3channel * opacity)).astype(np.uint8)
                    
                    # Draw landmarks (optional - you might want to remove these for the final video)
                    #mp_drawing.draw_landmarks(original_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                    #mp_drawing.draw_landmarks(original_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                    #mp_drawing.draw_landmarks(
                    #        original_image,
                    #        results.face_landmarks,
                    #        mp_holistic.FACEMESH_TESSELATION,
                    #        landmark_drawing_spec=None,
                    #        connection_drawing_spec=mp_drawing_styles
                    #        .get_default_face_mesh_tesselation_style())
                    #mp_drawing.draw_landmarks(
                    #        original_image,
                    #        results.pose_landmarks,
                    #        mp_holistic.POSE_CONNECTIONS,
                    #        landmark_drawing_spec=mp_drawing_styles.
                    #        get_default_pose_landmarks_style())
                    
                    # Save time series data
                    samplebody = listpostions(results.pose_landmarks)
                    samplehands = listpostions([results.left_hand_landmarks, results.right_hand_landmarks])
                    sampleface = listpostions(results.face_landmarks)
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    tsbody.append(samplebody)
                    tshands.append(samplehands)
                    tsface.append(sampleface)
                    
                if np.all(results.face_landmarks) == None:
                    original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    samplebody = [np.nan for x in range(len(markerxyzbody)-1)]
                    samplehands = [np.nan for x in range(len(markerxyzhands)-1)]
                    sampleface = [np.nan for x in range(len(markerxyzface)-1)]
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    tsbody.append(samplebody)
                    tshands.append(samplehands)
                    tsface.append(sampleface)
                
                cv2.imshow("resizedimage", original_image)
                out.write(original_image)
                time = time+(1000/samplerate)
                
            if cv2.waitKey(1) == 27:
                break
            if ret == False:
                break

    out.release()
    capture.release()
    cv2.destroyAllWindows()
    
    # Write CSV files
    filebody = open(outtputf_ts + vidf[:-4]+'_body.csv', 'w+', newline ='')
    with filebody:    
        write = csv.writer(filebody)
        write.writerows(tsbody)
        
    filehands = open(outtputf_ts + vidf[:-4]+'_hands.csv', 'w+', newline ='')
    with filehands:
        write = csv.writer(filehands)
        write.writerows(tshands)
        
    fileface = open(outtputf_ts + vidf[:-4]+'_face.csv', 'w+', newline ='')
    with fileface:    
        write = csv.writer(fileface)
        write.writerows(tsface)

print("Done with processing all folders; go look in your output folders!")

We will now process video:
DOLFIJN.mp4
This is video number 0 of 6 videos in total
We will now process video:
ETEN.mp4
This is video number 1 of 6 videos in total
We will now process video:
NULL.mp4
This is video number 2 of 6 videos in total
We will now process video:
OCHTEND.mp4
This is video number 3 of 6 videos in total
We will now process video:
OLIFANT.mp4
This is video number 4 of 6 videos in total
We will now process video:
RIETJE.mp4
This is video number 5 of 6 videos in total
Done with processing all folders; go look in your output folders!


In [14]:
# do you want to apply masking?
masking = True
blur_kernel_size = 111  # Adjust this value to change blur intensity
opacity = 1  # 0.0 is fully transparent, 1.0 is fully opaque

# Mouth landmarks for masking
MOUTH_LANDMARKS = [192, 206, 2, 426, 436, 434, 431, 211]

# We will now loop over all the videos that are present in the video file
for vidf in vfiles:
    print("We will now process video:")
    print(vidf)
    print("This is video number " + str(vfiles.index(vidf))+ " of " + str(len(vfiles)) + " videos in total")
    
    videoname = vidf
    videoloc = inputfol + videoname
    capture = cv2.VideoCapture(videoloc)
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    samplerate = capture.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(outputf_mask+'handm_'+videoname, fourcc, 
                         fps = samplerate, frameSize = (int(frameWidth), int(frameHeight)))

    time = 0
    tsbody = [markerxyzbody]
    tshands = [markerxyzhands]
    tsface = [markerxyzface]
    
    with mp_holistic.Holistic(
            static_image_mode=False, enable_segmentation=True, refine_face_landmarks=True) as holistic:
        while (True):
            ret, image = capture.read()
            if ret == True:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)
                
                h, w, c = image.shape
                if np.all(results.face_landmarks) != None:
                    original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    
                    if masking:
                        # Create mask for mouth area
                        mouth_mask = np.zeros((h, w), dtype=np.uint8)
                        landmarks = results.face_landmarks.landmark
                        
                        # Get mouth area points
                        mouth_points = np.array([(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) 
                                               for idx in MOUTH_LANDMARKS], dtype=np.int32)
                        
                        # Fill mouth polygon
                        cv2.fillPoly(mouth_mask, [mouth_points], 255)
                        
                        # Create hand mask
                        hand_mask = np.zeros((h, w), dtype=np.uint8)
                        
                        # Draw hands on the mask
                        if results.left_hand_landmarks:
                            hand_points = []
                            for landmark in results.left_hand_landmarks.landmark:
                                x = int(landmark.x * w)
                                y = int(landmark.y * h)
                                hand_points.append((x, y))
                            if len(hand_points) > 0:
                                hull = cv2.convexHull(np.array(hand_points))
                                cv2.fillConvexPoly(hand_mask, hull, 255)
                                
                        if results.right_hand_landmarks:
                            hand_points = []
                            for landmark in results.right_hand_landmarks.landmark:
                                x = int(landmark.x * w)
                                y = int(landmark.y * h)
                                hand_points.append((x, y))
                            if len(hand_points) > 0:
                                hull = cv2.convexHull(np.array(hand_points))
                                cv2.fillConvexPoly(hand_mask, hull, 255)
                        
                        # Dilate hand mask to create a more natural coverage
                        kernel = np.ones((15,15), np.uint8)
                        hand_mask = cv2.dilate(hand_mask, kernel, iterations=1)
                        
                        # Subtract hand area from mouth mask
                        mouth_mask = cv2.subtract(mouth_mask, hand_mask)
                        
                        # Apply Gaussian blur to the mouth area
                        blur_region = cv2.GaussianBlur(original_image, (blur_kernel_size, blur_kernel_size), 0)
                        
                        # Blend original and blurred image based on mask and opacity
                        mask_3channel = cv2.cvtColor(mouth_mask, cv2.COLOR_GRAY2BGR) / 255.0
                        original_image = (original_image * (1 - mask_3channel * opacity) + 
                                        blur_region * (mask_3channel * opacity)).astype(np.uint8)
                    
                    # Save time series data
                    samplebody = listpostions(results.pose_landmarks)
                    samplehands = listpostions([results.left_hand_landmarks, results.right_hand_landmarks])
                    sampleface = listpostions(results.face_landmarks)
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    tsbody.append(samplebody)
                    tshands.append(samplehands)
                    tsface.append(sampleface)
                    
                if np.all(results.face_landmarks) == None:
                    original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    samplebody = [np.nan for x in range(len(markerxyzbody)-1)]
                    samplehands = [np.nan for x in range(len(markerxyzhands)-1)]
                    sampleface = [np.nan for x in range(len(markerxyzface)-1)]
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    tsbody.append(samplebody)
                    tshands.append(samplehands)
                    tsface.append(sampleface)
                
                cv2.imshow("resizedimage", original_image)
                out.write(original_image)
                time = time+(1000/samplerate)
                
            if cv2.waitKey(1) == 27:
                break
            if ret == False:
                break

    out.release()
    capture.release()
    cv2.destroyAllWindows()
    
    # Write CSV files
    filebody = open(outtputf_ts + vidf[:-4]+'_body.csv', 'w+', newline ='')
    with filebody:    
        write = csv.writer(filebody)
        write.writerows(tsbody)
        
    filehands = open(outtputf_ts + vidf[:-4]+'_hands.csv', 'w+', newline ='')
    with filehands:
        write = csv.writer(filehands)
        write.writerows(tshands)
        
    fileface = open(outtputf_ts + vidf[:-4]+'_face.csv', 'w+', newline ='')
    with fileface:    
        write = csv.writer(fileface)
        write.writerows(tsface)

print("Done with processing all folders; go look in your output folders!")

We will now process video:
DOLFIJN.mp4
This is video number 0 of 6 videos in total
We will now process video:
ETEN.mp4
This is video number 1 of 6 videos in total
We will now process video:
NULL.mp4
This is video number 2 of 6 videos in total
We will now process video:
OCHTEND.mp4
This is video number 3 of 6 videos in total
We will now process video:
OLIFANT.mp4
This is video number 4 of 6 videos in total
We will now process video:
RIETJE.mp4
This is video number 5 of 6 videos in total
Done with processing all folders; go look in your output folders!


In [17]:
# do you want to apply masking?
masking = True
blur_kernel_size = 111  # Adjust this value to change blur intensity
opacity = 1  # 0.0 is fully transparent, 1.0 is fully opaque

# Mouth landmarks for masking
MOUTH_LANDMARKS = [192, 206, 2, 426, 436, 434, 431, 211]

# We will now loop over all the videos that are present in the video file
for vidf in vfiles:
    print("We will now process video:")
    print(vidf)
    print("This is video number " + str(vfiles.index(vidf))+ " of " + str(len(vfiles)) + " videos in total")
    
    videoname = vidf
    videoloc = inputfol + videoname
    capture = cv2.VideoCapture(videoloc)
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    samplerate = capture.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(outputf_mask+'handv3_'+videoname, fourcc, 
                         fps = samplerate, frameSize = (int(frameWidth), int(frameHeight)))

    time = 0
    tsbody = [markerxyzbody]
    tshands = [markerxyzhands]
    tsface = [markerxyzface]
    
    with mp_holistic.Holistic(
            static_image_mode=False, enable_segmentation=True, refine_face_landmarks=True) as holistic:
        while (True):
            ret, image = capture.read()
            if ret == True:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)
                
                h, w, c = image.shape
                if np.all(results.face_landmarks) != None:
                    original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    
                    if masking:
                        # Create mask for mouth area
                        mouth_mask = np.zeros((h, w), dtype=np.uint8)
                        landmarks = results.face_landmarks.landmark
                        
                        # Get mouth area points
                        mouth_points = np.array([(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) 
                                               for idx in MOUTH_LANDMARKS], dtype=np.int32)
                        
                        # Fill mouth polygon
                        cv2.fillPoly(mouth_mask, [mouth_points], 255)
                        
                        # Create hand mask
                        hand_mask = np.zeros((h, w), dtype=np.uint8)
                        
                        # Draw hands on the mask
                        if results.left_hand_landmarks:
                            hand_points = []
                            for landmark in results.left_hand_landmarks.landmark:
                                x = int(landmark.x * w)
                                y = int(landmark.y * h)
                                hand_points.append((x, y))
                            if len(hand_points) > 0:
                                hull = cv2.convexHull(np.array(hand_points))
                                cv2.fillConvexPoly(hand_mask, hull, 255)
                                
                        if results.right_hand_landmarks:
                            hand_points = []
                            for landmark in results.right_hand_landmarks.landmark:
                                x = int(landmark.x * w)
                                y = int(landmark.y * h)
                                hand_points.append((x, y))
                            if len(hand_points) > 0:
                                hull = cv2.convexHull(np.array(hand_points))
                                cv2.fillConvexPoly(hand_mask, hull, 255)
                        
                        # Create a more precise hand mask with minimal dilation
                        kernel = np.ones((5,5), np.uint8)
                        hand_mask = cv2.dilate(hand_mask, kernel, iterations=1)
                        
                        # Smooth the edges of the hand mask
                        hand_mask = cv2.GaussianBlur(hand_mask, (3,3), 0)
                        _, hand_mask = cv2.threshold(hand_mask, 127, 255, cv2.THRESH_BINARY)
                        
                        # First apply the mouth blur
                        blur_region = cv2.GaussianBlur(original_image, (blur_kernel_size, blur_kernel_size), 0)
                        mask_3channel = cv2.cvtColor(mouth_mask, cv2.COLOR_GRAY2BGR) / 255.0
                        blurred_image = (original_image * (1 - mask_3channel * opacity) + 
                                       blur_region * (mask_3channel * opacity)).astype(np.uint8)
                        
                        # Then overlay the hands
                        hand_mask_3channel = cv2.cvtColor(hand_mask, cv2.COLOR_GRAY2BGR) / 255.0
                        original_image = (blurred_image * (1 - hand_mask_3channel) + 
                                        original_image * hand_mask_3channel).astype(np.uint8)
                    
                    # Save time series data
                    samplebody = listpostions(results.pose_landmarks)
                    samplehands = listpostions([results.left_hand_landmarks, results.right_hand_landmarks])
                    sampleface = listpostions(results.face_landmarks)
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    tsbody.append(samplebody)
                    tshands.append(samplehands)
                    tsface.append(sampleface)
                    
                if np.all(results.face_landmarks) == None:
                    original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    samplebody = [np.nan for x in range(len(markerxyzbody)-1)]
                    samplehands = [np.nan for x in range(len(markerxyzhands)-1)]
                    sampleface = [np.nan for x in range(len(markerxyzface)-1)]
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    tsbody.append(samplebody)
                    tshands.append(samplehands)
                    tsface.append(sampleface)
                
                cv2.imshow("resizedimage", original_image)
                out.write(original_image)
                time = time+(1000/samplerate)
                
            if cv2.waitKey(1) == 27:
                break
            if ret == False:
                break

    out.release()
    capture.release()
    cv2.destroyAllWindows()
    
    # Write CSV files
    filebody = open(outtputf_ts + vidf[:-4]+'_body.csv', 'w+', newline ='')
    with filebody:    
        write = csv.writer(filebody)
        write.writerows(tsbody)
        
    filehands = open(outtputf_ts + vidf[:-4]+'_hands.csv', 'w+', newline ='')
    with filehands:
        write = csv.writer(filehands)
        write.writerows(tshands)
        
    fileface = open(outtputf_ts + vidf[:-4]+'_face.csv', 'w+', newline ='')
    with fileface:    
        write = csv.writer(fileface)
        write.writerows(tsface)

print("Done with processing all folders; go look in your output folders!")

We will now process video:
DOLFIJN.mp4
This is video number 0 of 6 videos in total
We will now process video:
ETEN.mp4
This is video number 1 of 6 videos in total
We will now process video:
NULL.mp4
This is video number 2 of 6 videos in total
We will now process video:
OCHTEND.mp4
This is video number 3 of 6 videos in total
We will now process video:
OLIFANT.mp4
This is video number 4 of 6 videos in total
We will now process video:
RIETJE.mp4
This is video number 5 of 6 videos in total
Done with processing all folders; go look in your output folders!
